# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [52]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


In [53]:
# Inspect the first few rows of the data
adult_dt.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [54]:
from sklearn.model_selection import train_test_split

# Create a dataframe for X with all features columns while excluding 'income'
X = adult_dt.drop(columns=['income'])

# Create a dataframe for y
y = adult_dt['income']

# Split the data into training and testing sets (70% training, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Display the shapes of the training and test sets
print(f'X_train shape:', X_train.shape)
print(f'X_test shape:', X_test.shape)
print(f'y_train shape:', y_train.shape)
print(f'y_test shape', y_test.shape)



X_train shape: (22792, 14)
X_test shape: (9769, 14)
y_train shape: (22792,)
y_test shape (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state of the splitting function is the seed for the random number generator that is used to split the data into training and testing sets. This ensures that the split is reproducible, which is important because it means that the data is split the same way each time we run the code. This is useful if we are trying to debug for errors and can isolate the issue, and it also allows us to compare models due to differences in the models rather than the data. 

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [55]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer

# Define the numerical and categorical features
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define preprocessing for numerical features
numerical_transformer = Pipeline(steps=[
    ('KNNImputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('RobustScaler', RobustScaler())
])

# Define preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('SimpleImputer', SimpleImputer(strategy='most_frequent')),
    ('OneHotEncoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
])

# Combine both pipelines using a ColumnTransformer
column_transformer = ColumnTransformer(
    transformers=[
        ('num_transforms', numerical_transformer, numerical_features),
        ('cat_transforms', categorical_transformer, categorical_features)
    ]
)



In [56]:
column_transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('KNNImputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('RobustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('SimpleImputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('OneHotEncoder',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Create a model pipeline with preprocessor and RandomForest Classifier
model_pipeline = Pipeline(steps=[
    ('preprocessing', column_transformer),
    ('classifier', RandomForestClassifier(random_state=42))
])

In [58]:
model_pipeline

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('KNNImputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('RobustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [59]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Define the performance metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Cross-validate with 5 folds
cv_results = cross_validate(
    model_pipeline,
    X_train,
    y_train,
    cv=5,
    scoring = scoring,
    return_train_score = True
)

# Report the training and validation results
cv_results_df = pd.DataFrame(cv_results)
print(cv_results_df)



c:\Users\Brian\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
c:\Users\Brian\miniconda3\envs\dsi_participant\lib\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
0  14.866998    0.287001          -0.357675           -0.081189      0.904384   
1  12.741000    0.210001          -0.369239           -0.081516      0.901079   
2  11.347000    0.241001          -0.375988           -0.081469      0.901378   
3  12.961001    0.244002          -0.356791           -0.082511      0.907250   
4  18.574005    0.417996          -0.380379           -0.081368      0.902299   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
0            1.0       0.850625        0.999945                0.774484   
1            1.0       0.850406        1.000000                0.771881   
2            1.0       0.854103        0.999945                0.776017   
3            1.0       0.859807        1.000000                0.782859   
4            1.0       0.856077        1.000000                0.776089   

   train_balanced_accuracy  
0                 0.999887  
1   

Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [60]:
# Sort the results by negative log loss of the test set
sorted_cv_results_df = cv_results_df.sort_values(by='test_neg_log_loss', ascending=True)

# Display the sorted results
print(sorted_cv_results_df)

    fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  18.574005    0.417996          -0.380379           -0.081368      0.902299   
2  11.347000    0.241001          -0.375988           -0.081469      0.901378   
1  12.741000    0.210001          -0.369239           -0.081516      0.901079   
0  14.866998    0.287001          -0.357675           -0.081189      0.904384   
3  12.961001    0.244002          -0.356791           -0.082511      0.907250   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.856077        1.000000                0.776089   
2            1.0       0.854103        0.999945                0.776017   
1            1.0       0.850406        1.000000                0.771881   
0            1.0       0.850625        0.999945                0.774484   
3            1.0       0.859807        1.000000                0.782859   

   train_balanced_accuracy  
4                 1.000000  
2   

Calculate the mean of each metric. 

In [61]:
mean_metrics = cv_results_df[
    [
    'train_neg_log_loss', 'test_neg_log_loss',
    'train_roc_auc', 'test_roc_auc',
    'train_accuracy','test_accuracy',
    'train_balanced_accuracy', 'test_balanced_accuracy'
    ]
].mean()

print(f'Mean Performance Metrics:')
print(mean_metrics.to_string())

Mean Performance Metrics:
train_neg_log_loss        -0.081610
test_neg_log_loss         -0.368014
train_roc_auc              1.000000
test_roc_auc               0.903278
train_accuracy             0.999978
test_accuracy              0.854204
train_balanced_accuracy    0.999955
test_balanced_accuracy     0.776266


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [67]:
# Fit the pipeline on the training data
model_pipeline.fit(X_train, y_train)

# Obtain predictions and probabilities for X_test
Y_pred = model_pipeline.predict(X_test)
Y_pred_proba = model_pipeline.predict_proba(X_test)

# Calculate the performance metrics
test_metrics = {
    'neg_log_loss': log_loss(y_test, Y_pred_proba),
    'roc_auc': roc_auc_score(y_test, Y_pred_proba[:, 1]),
    'accuracy': accuracy_score(y_test, Y_pred),
    'balanced_accuracy': balanced_accuracy_score(y_test, Y_pred)
}

# Display the results as a dictionary
print(test_metrics)




{'neg_log_loss': 0.39726506589398913, 'roc_auc': 0.8999828704291436, 'accuracy': 0.8551540587572934, 'balanced_accuracy': 0.7751631656838177}


In [69]:
print(f'Performance Metrics:')
for metric, value in test_metrics.items():
    print(f'{metric}: {value:.4f}')

Performance Metrics:
neg_log_loss: 0.3973
roc_auc: 0.9000
accuracy: 0.8552
balanced_accuracy: 0.7752


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

By loading the data and immediately recording the target variable, it enabled us to streamline the data preparation and make the target available for modeling. It also ensures consistency and that the code is simplified and easier to read. 

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.